- Прочитайте главы 7-8 из книги "Spark: The Definitive Guide".
- Загрузите датасеты по ссылкам:
    - https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2019
    - https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2020
    - https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2021
    - https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2022
- Выполните задания.

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
                    .master("local")\
                    .appName('SparkLab3')\
                    .getOrCreate()

# Загружаем наборы данных в один DataFrame.
tiktokData0 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/home/student/Downloads/TikTok_songs_csv/")

23/01/11 19:55:03 WARN Utils: Your hostname, student-VirtualBox resolves to a loopback address: 127.0.1.1; using 192.168.8.34 instead (on interface enp0s3)
23/01/11 19:55:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/11 19:55:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


№1: Добавьте столбец year целочисленного типа, который содержит год датасета для каждой строки данных. Выведите самые танцевальные песни из наборов данных вместе с полем year.

|          track_name|artist_name|year|danceability|
|--------------------|-----------|----|------------|
|  Dancing in My Room|   347aidan|2020|        0.98|
|                 Uno|    Ambjaay|2019|       0.978|
|I'm Looking for M...|   Memetown|2020|       0.977|
|           Hood Baby|       KBFR|2020|       0.974|
|              MICKEY| Lil Yachty|2019|       0.971|
|Muffins In The Fr...|      Tiagz|2020|       0.965|

<div style="text-align: center"> only showing top 6 rows </div>

In [14]:
from pyspark.sql.functions import input_file_name
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import desc
from pyspark.sql.types import IntegerType

tiktokData1 = tiktokData0\
  .withColumn("year", regexp_extract(input_file_name(), r"TikTok_songs_(\d+)\.csv", 1).cast("int"))

tiktokData1.sort(desc("danceability")).select("track_name", "artist_name", "year", "danceability").show(6)

+--------------------+-----------+----+------------+
|          track_name|artist_name|year|danceability|
+--------------------+-----------+----+------------+
|  Dancing in My Room|   347aidan|2020|        0.98|
|                 Uno|    Ambjaay|2019|       0.978|
|I'm Looking for M...|   Memetown|2020|       0.977|
|           Hood Baby|       KBFR|2020|       0.974|
|              MICKEY| Lil Yachty|2019|       0.971|
|Muffins In The Fr...|      Tiagz|2020|       0.965|
+--------------------+-----------+----+------------+
only showing top 6 rows



№2: Найдите исполнителей, которым сопоставлено более одного значения популярности. Выведите эти значения в виде списков.

|          artist|popularity|
|----------------|----------|
|            KYLE|  [65, 66]|
|Kero Kero Bonito|  [57, 56]|
|       Kesh Kesh|  [27, 26]|
|    Mahogany Lox|  [38, 37]|
|   Trevor Daniel|  [66, 65]|
|       blackbear|  [79, 80]|

In [15]:
from pyspark.sql.functions import col
from pyspark.sql.functions import collect_set
from pyspark.sql.functions import size

tiktokData2 = tiktokData1\
    .groupBy("artist_name").agg(collect_set("artist_pop").alias("popularity"))

tiktokData2.select(col("artist_name").alias("artist"), "popularity").where(
    size(col("popularity")) > 1
).show()

+----------------+----------+
|          artist|popularity|
+----------------+----------+
|            KYLE|  [66, 65]|
|Kero Kero Bonito|  [56, 57]|
|       Kesh Kesh|  [27, 26]|
|    Mahogany Lox|  [37, 38]|
|   Trevor Daniel|  [66, 65]|
|       blackbear|  [79, 80]|
+----------------+----------+



№3: Исправьте различающиеся значения популярности исполнителей. Замените их на максимальные значения из полученных в предыдущем задании массивов.

|          track_name|     artist_name|               album|artist_pop|
|--------------------|----------------|--------------------|----------|
|     hot girl bummer|       blackbear|     hot girl bummer|        80|
|Hey Julie! (feat....|            KYLE|Light of Mine (De...|        66|
|             Falling|   Trevor Daniel|             Falling|        66|
|             Falling|   Trevor Daniel|            Nicotine|        66|
|            Flamingo|Kero Kero Bonito|            Flamingo|        57|
|       Take Your Man|    Mahogany Lox|       Take Your Man|        38|
|              Vibin'|       Kesh Kesh|              Vibin'|        27|

<div style="text-align: center"> 968 </div>

In [16]:
from pyspark.sql.functions import array_max
from pyspark.sql.functions import when

tiktokData3 = tiktokData1\
    .drop("artist_pop")\
    .join(tiktokData2.withColumn("artist_pop", array_max(col("popularity"))).drop("popularity"),
          "artist_name", "left")\
    .select(tiktokData1.columns)

# Выводим различия между новым  и старым датафреймами
substractDf = tiktokData3\
    .subtract(tiktokData1)\
    .select("track_name", "artist_name", "album", "artist_pop")\
    .sort(desc("artist_pop"))

substractDf.show()
tiktokData3.count()

+--------------------+----------------+--------------------+----------+
|          track_name|     artist_name|               album|artist_pop|
+--------------------+----------------+--------------------+----------+
|     hot girl bummer|       blackbear|     hot girl bummer|        80|
|Hey Julie! (feat....|            KYLE|Light of Mine (De...|        66|
|             Falling|   Trevor Daniel|             Falling|        66|
|             Falling|   Trevor Daniel|            Nicotine|        66|
|            Flamingo|Kero Kero Bonito|            Flamingo|        57|
|       Take Your Man|    Mahogany Lox|       Take Your Man|        38|
|              Vibin'|       Kesh Kesh|              Vibin'|        27|
+--------------------+----------------+--------------------+----------+



968

№4: Сгруппируйте данные по названию трека, названию альбома, имени и популярности певца. Годы датасетов соберите в списки. Популярность треков возьмите из максимального значения. Для столбцов danceability, energy,   speechiness, acousticness, instrumentalness, liveness и valence возьмите среднее значение. Остальные столбцы отбросьте. Сколько получилось строк в сгруппированном DataFrame?

Полученный датафрейм сохраните в новый parquet файл. Это можно сделать методом:

```
df.write.format("parquet").mode("overwrite").save("path to file")
```

Результат при этом сохранится в новую директорию.
Другой способ:

```
df.toPandas().to_parquet("path to file")
```

Результат сохранится в один файл без создания директорий. Для этого способа понадобится библиотека pyarrow.
Установите ее с помощью команд:

```
source ~/Work/bin/activate
pip3 install pyarrow
```

In [18]:
from pyspark.sql.functions import expr
from pyspark.sql.functions import max
from pyspark.sql.functions import avg
from pyspark.sql.functions import size
from pyspark.sql.functions import sort_array

tiktokData4 = tiktokData3\
    .groupBy("track_name", "album", "artist_name", "artist_pop")\
    .agg(collect_set(col("year")).alias("years"),
        max(col("track_pop")).alias("track_pop"),
        avg(col("danceability")).alias("danceability"),
        avg(col("energy")).alias("energy"),
        avg(col("speechiness")).alias("speechiness"),
        avg(col("acousticness")).alias("acousticness"),
        avg(col("instrumentalness")).alias("instrumentalness"),
        avg(col("liveness")).alias("liveness"),
        avg(col("valence")).alias("valence"))

tiktokData4.show()
tiktokData4.count()
tiktokData4.write.format("parquet").mode("overwrite").save("task_4")

+--------------------+--------------------+--------------------+----------+------------+---------+------------+------+-----------+------------+----------------+--------+-------+
|          track_name|               album|         artist_name|artist_pop|       years|track_pop|danceability|energy|speechiness|acousticness|instrumentalness|liveness|valence|
+--------------------+--------------------+--------------------+----------+------------+---------+------------+------+-----------+------------+----------------+--------+-------+
|"Galactic Mermaid...|"Galactic Mermaid...|            Kuraiinu|        42|      [2019]|        0|       0.621| 0.446|     0.0384|       0.736|             0.0|   0.613|   0.97|
|"Legs (From ""RuP...|"Legs (From ""RuP...|        Lucian Piane|        32|      [2019]|       44|         0.9|  0.72|      0.232|     0.00502|         6.26E-6|   0.543|  0.749|
|"More Than A Woma...|            Greatest|            Bee Gees|        75|      [2020]|       63|       0.632

№5: Отсортируйте песни по количеству вхождений в датасеты и популярности. Выведите их вместе со списками годов наборов данных.

|          track_name|               album|     artist_name|track_pop|               years|
|--------------------|--------------------|----------------|---------|--------------------|
|              Say So|            Hot Pink|        Doja Cat|       80|[2019, 2020, 2021...|
|      Wait a Minute!|        ARDIPITHECUS|          WILLOW|       86|  [2019, 2020, 2022]|
|       Electric Love|            Dopamine|           BØRNS|       82|  [2020, 2021, 2022]|
|           Play Date|Cry Baby (Deluxe ...|Melanie Martinez|       78|  [2020, 2021, 2022]|
|             ROXANNE|             ROXANNE|  Arizona Zervas|       78|  [2019, 2020, 2022]|
|Savage Love (Laxe...|Savage Love (Laxe...|       Jawsh 685|       77|  [2020, 2021, 2022]|

<div style="text-align: center"> only showing top 6 rows </div>

In [19]:
tiktokData5 = tiktokData4.orderBy(desc(size(col("years"))), desc("track_pop")).select(
    "track_name", "album", "artist_name", "track_pop", "years"
)

tiktokData5.show(6)

+--------------------+--------------------+----------------+---------+--------------------+
|          track_name|               album|     artist_name|track_pop|               years|
+--------------------+--------------------+----------------+---------+--------------------+
|              Say So|            Hot Pink|        Doja Cat|       80|[2022, 2019, 2020...|
|      Wait a Minute!|        ARDIPITHECUS|          WILLOW|       86|  [2022, 2019, 2020]|
|       Electric Love|            Dopamine|           BØRNS|       82|  [2022, 2020, 2021]|
|             ROXANNE|             ROXANNE|  Arizona Zervas|       78|  [2022, 2019, 2020]|
|           Play Date|Cry Baby (Deluxe ...|Melanie Martinez|       78|  [2022, 2020, 2021]|
|Savage Love (Laxe...|Savage Love (Laxe...|       Jawsh 685|       77|  [2022, 2020, 2021]|
+--------------------+--------------------+----------------+---------+--------------------+
only showing top 6 rows



№6: Найдите все треки с повторяющимися именами. Выполните их сортировку по названию. Выведите названия песен, названия альбомов, имена исполнителей и годы датасетов. Сколько строк в полученной таблице?

|          track_name|               album|   artist_name| years|
|--------------------|--------------------|--------------|------|
|"More Than A Woma...|How Can You Mend ...|      Bee Gees|[2022]|
|"More Than A Woma...|            Greatest|      Bee Gees|[2020]|
|223's (feat. 9lok...|    Melly vs. Melvin|     YNW Melly|[2019]|
|223's (feat. 9lok...|223's (feat. 9lok...|     YNW Melly|[2020]|
|           As It Was|           As It Was|Michael Shynes|[2022]|
|           As It Was|           As It Was|  Harry Styles|[2022]|

<div style="text-align: center"> only showing top 6 rows </div>

In [47]:
from pyspark.sql.functions import count

tiktokData6 = tiktokData5\
    .join(tiktokData5.groupBy("track_name").agg(count("*").alias("count")).where(col("count") > 1),
          "track_name")\
    .select(tiktokData5.track_name, "album", "artist_name", "years")\
    .orderBy(tiktokData5.track_name)

tiktokData6.show(6)
tiktokData6.count()

+--------------------+--------------------+--------------+------+
|          track_name|               album|   artist_name| years|
+--------------------+--------------------+--------------+------+
|"More Than A Woma...|            Greatest|      Bee Gees|[2020]|
|"More Than A Woma...|How Can You Mend ...|      Bee Gees|[2022]|
|223's (feat. 9lok...|223's (feat. 9lok...|     YNW Melly|[2020]|
|223's (feat. 9lok...|    Melly vs. Melvin|     YNW Melly|[2019]|
|           As It Was|           As It Was|Michael Shynes|[2022]|
|           As It Was|           As It Was|  Harry Styles|[2022]|
+--------------------+--------------------+--------------+------+
only showing top 6 rows



85